In [4]:
import numpy as np
import pandas as pd
import PIL
import matplotlib.patches as patches
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Dropout,Flatten,Conv2D,MaxPooling2D
from tensorflow.keras.preprocessing import image

In [7]:
attr_data_path = "list_attr_celeba.csv"
images_data_path = "C:/Users/foreh/Documents/celeba/greyscale"
images_data_size = 10000 

width = 178
height = 218
x_ = 100                            # new value of x
image_size_ratio = width / height    # dimensions ratio
y_ = int(image_size_ratio * x_)  
model = Sequential()
model.add(tf.keras.Input(shape=(height, width,1)))
model.add(tf.keras.layers.Normalization())

model.add(Conv2D(filters=16,kernel_size=(8,8),padding = 'valid'))
model.add(Conv2D(filters=16, kernel_size=(5,5), padding='valid'))
model.add(tf.keras.layers.Normalization())

model.add(Conv2D(filters=12, kernel_size=(4, 4), padding='same'))
model.add(MaxPooling2D(pool_size=2))

model.add(Dropout(0.2))

model.add(Conv2D(filters=16,kernel_size=(3,3),padding = 'same'))
model.add(Conv2D(filters=8,kernel_size=(2,2),padding = 'same'))
model.add(tf.keras.layers.Normalization())

model.add(Conv2D(filters=4, kernel_size=(3, 3), padding='same'))
model.add(MaxPooling2D(pool_size=2))

model.add(Dropout(0.2))

model.add(Conv2D(filters=16,kernel_size=(3,3),padding = 'same'))
model.add(Conv2D(filters=8,kernel_size=(2,2),padding = 'same'))

model.add(tf.keras.layers.Normalization())

model.add(Conv2D(filters=4, kernel_size=(3, 3), padding='same'))
model.add(MaxPooling2D(pool_size=4))

model.add(Conv2D(filters=16, kernel_size=(3,3)))
model.add(Conv2D(filters=16, kernel_size=(2,2)))
model.add(tf.keras.layers.MaxPooling2D(pool_size=3))

model.add(tf.keras.layers.Normalization())

# DOUBLED IT
model.add(Conv2D(filters=32,kernel_size=(3,3),padding = 'same'))
model.add(Conv2D(filters=8, kernel_size=(3,3), padding='same'))
model.add(MaxPooling2D(pool_size=2))

model.add(Conv2D(filters=16,kernel_size=(3,3),padding = 'same'))
model.add(Conv2D(filters=8,kernel_size=(2,2),padding = 'same'))
model.add(tf.keras.layers.Normalization())
# END DOUBLE

model.add(Flatten())
model.add(Dense(64))
model.add(Dense(48))
model.add(Dense(16))
model.add(Dense(4, activation='sigmoid'))
original_image_size = (width,height)
df_org = pd.read_csv(attr_data_path)

for i in range(1, 21):
    tf.keras.backend.clear_session()
    if (i > 1):
        model = tf.keras.models.load_model('cnn9_checkpoints/iter'+str(i-1))
        print("model loaded: iter"+str(i-1))
        print("training model "+str(i))
    df = df_org[(1+images_data_size*i):images_data_size*(i+1)]
    df = df.replace(-1,0)
    #print(df.head(5))
    images_data = list()
    for idx in range(df.shape[0]):
        path = "{}/{}".format(str(images_data_path),str(df.iloc[idx].image_id))
        image = PIL.Image.open(path)
        image_array = np.asarray(image)
        images_data.append(image_array)
    images_data = np.array(images_data)
    images = images_data # list of array of images
    labels = df[["image_id", "Bangs", "Straight_Hair", "Wavy_Hair", "Wearing_Hat"]]  # dataframe of image features
    train_images,test_images,train_labels,test_labels = train_test_split(images,labels,test_size=0.3,random_state=(i%6))
    y_test=test_labels.drop(["image_id"],axis=1)
    y_train=train_labels.drop(["image_id"],axis=1)

    X_test=test_images
    X_train=train_images
    #X_train = tf.expand_dims(X_train, axis=-1)
    print("Samples Training Set:",len(X_train))
    X_test, X_val, y_test, y_val=train_test_split(X_test, y_test,test_size=0.5,random_state=(i%5))
    #model.evaluate(X_val,y_val)
    print("Samples in Validation Data:",len(X_val))
    print("Samples in Test Data:",len(X_test))
    x_ = original_image_size[0]
    y_ = original_image_size[1]
    BATCH_SIZE = (2*(i%2+1))
    print("Batch size: " + str(BATCH_SIZE))
    if True or i%3 == 1:
          print("SGD")
          opt = tf.keras.optimizers.SGD(learning_rate=0.002)
          model.compile(optimizer = "sgd",loss="binary_crossentropy",metrics = [tf.keras.metrics.Precision(), tf.keras.metrics.Recall(), 'binary_accuracy'])
    else:
          print("ADAM")
          model.compile(optimizer = "adam",loss="binary_crossentropy",metrics = [tf.keras.metrics.Precision(), tf.keras.metrics.Recall(), 'binary_accuracy'])
    model.evaluate(X_val,y_val)
    training_process = model.fit(X_train,y_train,epochs = (5+i%2),validation_data = (X_val,y_val),batch_size=BATCH_SIZE,shuffle = True)
    model.save('cnn9_checkpoints/iter'+str(i))
    print("Model " + str(i) +"evaluating:")
    model.evaluate(X_val,y_val)
    tf.keras.backend.clear_session()

Samples Training Set: 6999
Samples in Validation Data: 1500
Samples in Test Data: 1500
Batch size: 4
SGD
47/47 [==============================] - 3s 47ms/step - loss: 0.8523 - precision: 0.1826 - recall: 0.4457 - binary_accuracy: 0.5108
Epoch 1/6
1750/1750 [==============================] - 52s 29ms/step - loss: 0.4362 - precision: 0.5322 - recall: 0.0906 - binary_accuracy: 0.8195 - val_loss: 0.4429 - val_precision: 0.6628 - val_recall: 0.0495 - val_binary_accuracy: 0.8128
Epoch 2/6
1750/1750 [==============================] - 50s 29ms/step - loss: 0.3946 - precision: 0.6215 - recall: 0.1732 - binary_accuracy: 0.8299 - val_loss: 0.4391 - val_precision: 0.4941 - val_recall: 0.4023 - val_binary_accuracy: 0.8063
Epoch 3/6
1750/1750 [==============================] - 51s 29ms/step - loss: 0.3723 - precision: 0.6484 - recall: 0.2288 - binary_accuracy: 0.8366 - val_loss: 0.3801 - val_precision: 0.8240 - val_recall: 0.0895 - val_binary_accuracy: 0.8217
Epoch 4/6
1750/1750 [===================

KeyboardInterrupt: 

In [ ]:
df = df_org[:images_data_size]
df = df.replace(-1,0)
#print(df.head(5))
images_data = list()
for idx in range(df.shape[0]):
    path = "{}/{}".format(str(images_data_path),str(df.iloc[idx].image_id))
    image = PIL.Image.open(path)
    image_array = np.asarray(image)
    images_data.append(image_array)
images_data = np.array(images_data)
def image_from_index(index, size=(original_image_size), path=images_data_path):
    path = "{}/{}".format(str(path),str(df.iloc[index].image_id))
    # to read the image
    image = PIL.Image.open(path).resize(size)
    image_array = np.asarray(image)

    return image_array
images = images_data # list of array of images
labels = df[["image_id", "Bangs", "Straight_Hair", "Wavy_Hair", "Wearing_Hat"]]  # dataframe of image features
train_images,test_images,train_labels,test_labels = train_test_split(images,labels,test_size=0.3,random_state=45)
y_test=test_labels.drop(["image_id"],axis=1)
y_train=train_labels.drop(["image_id"],axis=1)

X_test=test_images
X_train=train_images
#X_train = tf.expand_dims(X_train, axis=-1)
print("Samples Training Set:",len(X_train))
X_test, X_val, y_test, y_val=train_test_split(X_test, y_test,test_size=0.5,random_state=42)
for i in range(1, 10):
    model1 = tf.keras.models.load_model('cnn9_checkpoints/iter'+str(i))
    model1.evaluate(X_val,y_val)

In [8]:
df_org.columns

Index(['image_id', '5_o_Clock_Shadow', 'Arched_Eyebrows', 'Attractive',
       'Bags_Under_Eyes', 'Bald', 'Bangs', 'Big_Lips', 'Big_Nose',
       'Black_Hair', 'Blond_Hair', 'Blurry', 'Brown_Hair', 'Bushy_Eyebrows',
       'Chubby', 'Double_Chin', 'Eyeglasses', 'Goatee', 'Gray_Hair',
       'Heavy_Makeup', 'High_Cheekbones', 'Male', 'Mouth_Slightly_Open',
       'Mustache', 'Narrow_Eyes', 'No_Beard', 'Oval_Face', 'Pale_Skin',
       'Pointy_Nose', 'Receding_Hairline', 'Rosy_Cheeks', 'Sideburns',
       'Smiling', 'Straight_Hair', 'Wavy_Hair', 'Wearing_Earrings',
       'Wearing_Hat', 'Wearing_Lipstick', 'Wearing_Necklace',
       'Wearing_Necktie', 'Young'],
      dtype='object')